# Hello 3D Bouding Box Extraction

In [1]:
# Prerequisites & Dependencies
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Load Point Cloud with Pandas

In [2]:
pc = pd.read_csv("point_clouds_public/lidar_dbscan_clusters.xyz",
                 delimiter=';', names=['X','Y','Z','Label'], header=None)

pc['Label'] = pc['Label'].astype(int)
print("Shape: ", pc.shape)
pc.head()

Shape:  (47894, 4)


,X,Y,Z,Label
0,22.961000,0.068,0.986,1
1,23.191000,0.141,0.993,1
2,23.252001,0.178,0.995,1
3,23.313999,0.252,0.998,1
4,23.389000,0.327,1.000,1


### Prepare Clusters by Splitting Dataframe into Groups

In [3]:
segments = pc.groupby(['Label'])

### Extract Bounding Boxes for Clusters

In [6]:
CLUSTER_NUM = 3

cluster = segments.get_group(CLUSTER_NUM)[['X','Y','Z']]

# Create point cloud of one cluster
pc_cluster = o3d.geometry.PointCloud()
pc_cluster.points = o3d.utility.Vector3dVector(np.array(cluster))

# Bounding Box
bbox = pc_cluster.get_axis_aligned_bounding_box()
# Oriented Bounding Box
obbox = pc_cluster.get_oriented_bounding_box()


C:\Users\Jari\AppData\Local\Temp\ipykernel_35244\3057028746.py:3: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  cluster = segments.get_group(CLUSTER_NUM)[['X','Y','Z']]


### Sanity Check

In [11]:
print("Cluster min:\n", cluster.min())
print("Cluster max:\n", cluster.max())
print("Bounding Box:\n", bbox)

Cluster min:
 X    26.938999
Y     4.673000
Z    -1.226000
dtype: float64
Cluster max:
 X    29.139999
Y     7.995000
Z     1.214000
dtype: float64
Bounding Box:
 AxisAlignedBoundingBox: min: (26.939, 4.673, -1.226), max: (29.14, 7.995, 1.214)


### Visualize

In [12]:
bbox.color = (0.8, 0, 0)
obbox.color = (0, 0.8, 0)
o3d.visualization.draw_geometries([bbox, obbox, pc_cluster])

### Visualize

In [18]:
bboxes = []
max_points = 300
min_points = 40

for i in range(len(segments)):
    cluster = segments.get_group(i)[['X','Y','Z']]
    if len(cluster) > min_points and len(cluster) < max_points:
        bboxes.append(o3d.geometry.PointCloud(o3d.utility.Vector3dVector(np.array(cluster))).get_axis_aligned_bounding_box())
        bboxes[-1].color = (0, 0, 0.8)

C:\Users\Jari\AppData\Local\Temp\ipykernel_35244\2967485103.py:6: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  cluster = segments.get_group(i)[['X','Y','Z']]
C:\Users\Jari\AppData\Local\Temp\ipykernel_35244\2967485103.py:6: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  cluster = segments.get_group(i)[['X','Y','Z']]
C:\Users\Jari\AppData\Local\Temp\ipykernel_35244\2967485103.py:6: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  cluster = segments.get_group(i)[['X','Y','Z']]
C:\Users\Jari\AppData\Local\Temp\ipykernel_35244\2967485103.py:6

### Import Ground as Open3D object 

In [19]:
pc_ground = o3d.io.read_point_cloud("point_clouds_public/lidar_ground_inliers.ply", format='ply')
pc_ground.paint_uniform_color([0.8, 0.8, 0.8])

PointCloud with 75790 points.

### Convert Pandas point cloud to Open3D colored point cloud

In [20]:
pc_o3d = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(np.array(pc[['X','Y','Z']])))
labels = np.array(pc['Label'])
nr_clusters = max(labels)
print(f"Point cloud has {nr_clusters} clusters")

colors = plt.get_cmap("tab20")(labels / (nr_clusters if nr_clusters > 0 else 1))
colors[labels < 0] = 0
pc_o3d_colors  = o3d.utility.Vector3dVector(colors[:, :3])


Point cloud has 296 clusters


### Visualize

In [21]:
o3d.visualization.draw_geometries(bboxes+[pc_ground, pc_o3d])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


### Function

In [27]:
def BboxExtract(points, min_points, max_points, color=(0,0,0.8)):
    bboxes = []

    for i in range(len(segments)):
        cluster = segments.get_group(i)[['X','Y','Z']]
        if len(cluster) > min_points and len(cluster) < max_points:
            bboxes.append(o3d.geometry.PointCloud(o3d.utility.Vector3dVector(np.array(cluster))).get_axis_aligned_bounding_box())
            bboxes[-1].color = color

    return bboxes

### Obstacle Classification

In [28]:
t_0 = time.time()
small_bboxes = BboxExtract(segments, 4, 40, color=(0.8, 0, 0))
t_1 = time.time()
medium_bboxes = BboxExtract(segments, 40, 300, color=(0, 0.8, 0))
t_2 = time.time()
large_bboxes = BboxExtract(segments, 300, 2000, color=(0, 0, 0.8))
t_3 = time.time()

print(f"Time to compute small bboxes {t_1 - t_0} seconds")
print(f"Time to compute medium bboxes {t_2 - t_1} seconds")
print(f"Time to compute large bboxes {t_3 - t_2} seconds")

o3d.visualization.draw_geometries(small_bboxes+medium_bboxes+large_bboxes+[pc_ground, pc_o3d])


C:\Users\Jari\AppData\Local\Temp\ipykernel_35244\2507222990.py:5: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  cluster = segments.get_group(i)[['X','Y','Z']]
C:\Users\Jari\AppData\Local\Temp\ipykernel_35244\2507222990.py:5: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  cluster = segments.get_group(i)[['X','Y','Z']]
C:\Users\Jari\AppData\Local\Temp\ipykernel_35244\2507222990.py:5: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  cluster = segments.get_group(i)[['X','Y','Z']]
C:\Users\Jari\AppData\Local\Temp\ipykernel_35244\2507222990.py:5

Time to compute small bboxes 0.09205269813537598 seconds
Time to compute medium bboxes 0.06949424743652344 seconds
Time to compute large bboxes 0.11555624008178711 seconds
